In [1]:
!hostname

cf53b32e0db4


In [2]:
!jupyter --version

4.4.0


https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

potential additional work:
* add strace capture
 * https://github.com/blapid/strace2py
 * https://python-ptrace.readthedocs.io/en/latest/syscall.html
 * https://github.com/dirtyharrycallahan/pystrace

In [3]:
!pip install psutil

In [4]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
print('pandas',pandas.__version__)
import matplotlib.pyplot as plt

pandas 0.23.4


# one-time system details

In [5]:
print(psutil.cpu_freq())

None


In [6]:
pid = os.getpid()
print(pid)

195


In [7]:
py = psutil.Process(pid)
print(py)

psutil.Process(pid=195, name='python', started='11:20:12')


In [8]:
py.name()

'python'

In [9]:
py.exe()

'/opt/conda/bin/python3.6'

In [10]:
py.cwd()

'/home/jovyan/jupyter_notebooks/week_09_timeseries/host_streaming_stats'

In [11]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json']

# system performance

In [12]:
# gives a single float value
psutil.cpu_percent(percpu=True)

[64.3, 56.6]

In [13]:
psutil.cpu_times_percent(percpu=True)

[scputimes(user=46.4, nice=0.0, system=19.2, idle=34.4, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0),
 scputimes(user=31.1, nice=0.0, system=25.8, idle=43.0, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0)]

In [14]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

{'user': 5.0, 'nice': 0.0, 'system': 1.0, 'idle': 0.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}
{'user': 1.0, 'nice': 0.0, 'system': 2.0, 'idle': 2.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}


In [15]:
# gives an object with many fields
psutil.virtual_memory()

svmem(total=2096164864, available=1306329088, percent=37.7, used=622710784, free=97165312, active=720723968, inactive=1165774848, buffers=67117056, cached=1309171712, shared=544768, slab=84848640)

In [16]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 2096164864,
 'available': 1306202112,
 'percent': 37.7,
 'used': 622854144,
 'free': 97038336,
 'active': 720728064,
 'inactive': 1165774848,
 'buffers': 67117056,
 'cached': 1309155328,
 'shared': 544768,
 'slab': 84848640}

In [17]:
dict(psutil.swap_memory()._asdict())

{'total': 1073737728,
 'used': 335872,
 'free': 1073401856,
 'percent': 0.0,
 'sin': 0,
 'sout': 335872}

In [18]:
dict(psutil.cpu_stats()._asdict())

{'ctx_switches': 1423720,
 'interrupts': 682192,
 'soft_interrupts': 289842,
 'syscalls': 0}

In [ ]:
psutil.disk_io_counters(perdisk=True).keys()

dict_keys(['loop0', 'loop1', 'loop2', 'loop3', 'loop4', 'loop5', 'loop6', 'loop7', 'nbd0', 'nbd1', 'nbd2', 'nbd3', 'nbd4', 'nbd5', 'nbd6', 'nbd7', 'nbd8', 'nbd9', 'nbd10', 'nbd11', 'nbd12', 'nbd13', 'nbd14', 'nbd15', 'sda', 'sda1', 'sr0', 'sr1', 'sr2'])

In [ ]:
dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict())

{'read_count': 0,
 'write_count': 0,
 'read_bytes': 0,
 'write_bytes': 0,
 'read_time': 0,
 'write_time': 0,
 'read_merged_count': 0,
 'write_merged_count': 0,
 'busy_time': 0}

In [ ]:
psutil.sensors_temperatures()

{}

In [ ]:
psutil.sensors_fans()

{}

In [ ]:
psutil.sensors_battery()

### put everything into a single row

In [ ]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input, got",type(row))
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [ ]:
os.getpid()

195

In [ ]:
type(os.getpid())

int

In [ ]:
py = psutil.Process(os.getpid())
print(py)

psutil.Process(pid=195, name='python', started='11:20:12')


In [ ]:
dict(py.memory_info()._asdict())

{'rss': 110874624,
 'vms': 855003136,
 'shared': 31784960,
 'text': 2232320,
 'lib': 0,
 'data': 189743104,
 'dirty': 0}

In [ ]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

memory use: 0.10326004028320312 GB


In [ ]:
dict(py.memory_full_info()._asdict())

{'rss': 110874624,
 'vms': 855003136,
 'shared': 31784960,
 'text': 2232320,
 'lib': 0,
 'data': 189759488,
 'dirty': 0,
 'uss': 80478208,
 'pss': 89944064,
 'swap': 0}

In [ ]:
py.cpu_percent()

0.0

In [ ]:
dict(py.cpu_times()._asdict())

{'user': 1.84, 'system': 0.66, 'children_user': 1.96, 'children_system': 1.26}

In [ ]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json']

In [ ]:
dict(py.num_ctx_switches()._asdict())

{'voluntary': 1058, 'involuntary': 1896}

In [ ]:
dict(py.io_counters()._asdict())

{'read_count': 6421,
 'write_count': 1116,
 'read_bytes': 2351104,
 'write_bytes': 4096,
 'read_chars': 29431926,
 'write_chars': 923277}

In [ ]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input for row, got",type(row))
        return None
    if not isinstance(pid,int):
        raise TypeError("ERROR: expected a int as input for pid, got",type(pid))
        return None
    try:
        mem_dict=dict(psutil.Process(pid).memory_full_info()._asdict())
    except NoSuchProcess:
        return row
    for k,v in mem_dict.items():
        row['process memory: '+k]=v

    try:
        cpu_times_dict=dict(psutil.Process(pid).cpu_times()._asdict())
    except NoSuchProcess:
        return row
    for k,v in cpu_times_dict.items():
        row['process cpu times: '+k]=v

    try:
        context_switches_dict=dict(psutil.Process(pid).num_ctx_switches()._asdict())
    except NoSuchProcess:
        return row        
    for k,v in context_switches_dict.items():
        row['process number context switches: '+k]=v

    try:
        io_counter_dict = dict(psutil.Process(pid).io_counters()._asdict())
    except NoSuchProcess:
        return row
    for k,v in io_counter_dict.items():
        row['process io counters: '+k]=v
    return row

# gather data and create plots

In [ ]:
polling_period_in_seconds = 1
monitoring_duration_in_seconds=10

In [ ]:
list_of_rows=[]
start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

elapsed time: 10.644303321838379 seconds


In [ ]:
print(df.shape)
df.head()

(10, 328)


,CPU 1 times guest,CPU 1 times guest_nice,CPU 1 times idle,CPU 1 times iowait,CPU 1 times irq,CPU 1 times nice,CPU 1 times softirq,CPU 1 times steal,CPU 1 times system,CPU 1 times user,...,system virtual memory buffers,system virtual memory cached,system virtual memory free,system virtual memory inactive,system virtual memory percent,system virtual memory shared,system virtual memory slab,system virtual memory total,system virtual memory used,time in unix seconds
0,0.0,0.0,9.9,0.0,0.0,0.0,3.0,0.0,39.6,47.5,...,67117056,1309175808,95649792,1165791232,37.8,544768,84877312,2096164864,624222208,1.552908e+09
1,0.0,0.0,97.1,0.0,0.0,0.0,0.0,0.0,1.0,2.0,...,67117056,1309200384,95522816,1165791232,37.8,544768,84905984,2096164864,624324608,1.552908e+09
2,0.0,0.0,95.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,...,67129344,1309200384,95522816,1165791232,37.8,544768,84905984,2096164864,624312320,1.552908e+09
3,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,67129344,1309196288,95522816,1165787136,37.8,544768,84905984,2096164864,624316416,1.552908e+09
4,0.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,67129344,1309196288,95531008,1165787136,37.8,544768,84869120,2096164864,624308224,1.552908e+09


In [ ]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [ ]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        raise TypeError('ERROR: expected input of type dataframe, got',type(df))
        return None
    if not isinstance(col_name,str):
        raise TypeError('ERROR: expected input of type str, got',type(col_name))
        return None
    plt.plot_date(x=df['datetime'],y=df[col_name])
    plt.xticks(rotation=40)
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

works even when the kernel of the other notebook is restarted

detects when new newbook kernels being added

https://github.com/iiSeymour/nbtop/blob/master/nbtop/main.py

### get a list of all active sessions

In [ ]:
r = requests.get('http://localhost:8888/api/sessions')
r.status_code

200

In [ ]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
 'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
 'name': 'realtime_data_matplotlib.ipynb',
 'type': 'notebook',
 'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
  'name': 'python3',
  'last_activity': '2019-03-18T11:02:46.614172Z',
  'execution_state': 'idle',
  'connections': 1},
 'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb'}}

### from json returned from API, we can extract the notebook kernel IDs

In [ ]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

notebook kernel IDs:
b1c1ad07-ecaa-4181-8258-f9a30ab8eafc
e5598466-2d75-4a55-9b0c-cf7d072159ba
047727fb-9390-40aa-ab15-ad3be786761b
154e7a2a-e9e3-44d8-aece-7d2a915156d0


### get a list of all pids

In [ ]:
print('all processes running:')
for p in psutil.process_iter():
    print(p)

all processes running:
psutil.Process(pid=1, name='tini', started='11:02:15')
psutil.Process(pid=6, name='jupyter-lab', started='11:02:16')
psutil.Process(pid=26, name='python', started='11:02:41')
psutil.Process(pid=144, name='python', started='11:12:43')
psutil.Process(pid=162, name='python', started='11:12:56')
psutil.Process(pid=195, name='python', started='11:20:12')


### get pid for each notebook

In [ ]:
def get_all_kernel_pids():
    dict_of_jupyter_kernel_pids = {}
    for p in psutil.process_iter():
        list_of_commands_per_process = psutil.Process(p.pid).cmdline()
#        print(list_of_commands_per_process)
        if ('ipykernel_launcher' in list_of_commands_per_process):
            dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        
    return dict_of_jupyter_kernel_pids

In [ ]:
dict_of_jupyter_kernel_pids = get_all_kernel_pids()
dict_of_jupyter_kernel_pids

{26: '/home/jovyan/.local/share/jupyter/runtime/kernel-b1c1ad07-ecaa-4181-8258-f9a30ab8eafc.json',
 144: '/home/jovyan/.local/share/jupyter/runtime/kernel-e5598466-2d75-4a55-9b0c-cf7d072159ba.json',
 162: '/home/jovyan/.local/share/jupyter/runtime/kernel-154e7a2a-e9e3-44d8-aece-7d2a915156d0.json',
 195: '/home/jovyan/.local/share/jupyter/runtime/kernel-047727fb-9390-40aa-ab15-ad3be786761b.json'}

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [ ]:
def add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts):
    if not isinstance(dict_of_jupyter_kernel_pids,dict):
        raise TypeError("ERROR: expected a dict as input for dict_of_jupyter_kernel_pids, got",type(dict_of_jupyter_kernel_pids))
        return None
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
        for notebook_indx in range(len(list_of_kernel_dicts)):
            if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#                print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
                list_of_kernel_dicts[notebook_indx]['pid']=pid
    return list_of_kernel_dicts

In [ ]:
list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)

In [ ]:
list_of_kernel_dicts

[{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
   'name': 'python3',
   'last_activity': '2019-03-18T11:02:46.614172Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
   'name': 'realtime_data_matplotlib.ipynb'},
  'pid': 26},
 {'id': 'c1db6eca-dcfe-4a4f-a083-3c19eae27a82',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': 'analysis_of_static_timeseries.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'e5598466-2d75-4a55-9b0c-cf7d072159ba',
   'name': 'python3',
   'last_activity': '2019-03-18T11:13:18.528379Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {

### which kernels are not this notebook?

In [ ]:
def kernel_dicts_other_notebooks(list_of_kernel_dicts):
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    list_of_kernel_dicts_for_other_notebooks=[]
    for kernel_dict in list_of_kernel_dicts:
        if (str(os.getpid())!=str(kernel_dict['pid'])):
            list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)
    return list_of_kernel_dicts_for_other_notebooks

In [ ]:
list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)
list_of_kernel_dicts_for_other_notebooks

[{'id': '1b5ea68c-d427-40c8-b1cf-f5531e992405',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'b1c1ad07-ecaa-4181-8258-f9a30ab8eafc',
   'name': 'python3',
   'last_activity': '2019-03-18T11:02:46.614172Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
   'name': 'realtime_data_matplotlib.ipynb'},
  'pid': 26},
 {'id': 'c1db6eca-dcfe-4a4f-a083-3c19eae27a82',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': 'analysis_of_static_timeseries.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'e5598466-2d75-4a55-9b0c-cf7d072159ba',
   'name': 'python3',
   'last_activity': '2019-03-18T11:13:18.528379Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {

In [ ]:
def get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks,
                          url_for_api_sessions='http://localhost:8888/api/sessions'):
    if not isinstance(list_of_kernel_dicts_for_other_notebooks,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts_for_other_notebooks, got",type(list_of_kernel_dicts_for_other_notebooks))
        return None

    list_of_kernel_dicts = requests.get(url_for_api_sessions).json()
    dict_of_jupyter_kernel_pids = get_all_kernel_pids()
    list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)
    current_list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)

    list_of_pids_in_previous=[]
    for old_dict in list_of_kernel_dicts_for_other_notebooks:
        list_of_pids_in_previous.append(old_dict['pid'])
        for curr_dict in current_list_of_kernel_dicts_for_other_notebooks:
            if (curr_dict['pid'] not in list_of_pids_in_previous):
                list_of_kernel_dicts_for_other_notebooks.append(curr_dict)
    return list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [ ]:
monitoring_duration_in_seconds=60*10

In [ ]:
nb_dict={}
list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids([])

start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    system_row = row_for_system_performance(row)

    # detect new notebook kernels
    list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks)
    
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        try:
            nb_dict[other_nb['pid']].append(row)
        except KeyError: # new pid for new kernel
            nb_dict[other_nb['pid']] = []
            nb_dict[other_nb['pid']].append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

In [ ]:
nb_df.keys()

In [ ]:
for indx in range(len(list_of_kernel_dicts_for_other_notebooks)):
    nb_pid = list_of_kernel_dicts_for_other_notebooks[indx]['pid']
    list_of_kernel_dicts_for_other_notebooks[indx]['df']=nb_df[nb_pid]

In [ ]:
with open("single_process_df_other_notebook_pid_"+str(time.time())+".pkl", 'wb') as handle:
    pickle.dump(list_of_kernel_dicts_for_other_notebooks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#   list_of_kernel_dicts_for_other_notebooks = pickle.load(handle)